In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

import os
# from pathlib import Path
from time import perf_counter
# startTime = perf_counter()


In [4]:

def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open(f"{os.path.abspath('')}/../data/ml-100k/u.ITEM", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

nameDict = spark.sparkContext.broadcast(loadMovieNames())

# Create schema when reading u.data
schema = StructType([
    StructField("userID", IntegerType(), True),
    StructField("movieID", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", LongType(), True)
])

In [5]:
spark

In [6]:

# Load up movie data as dataframe
moviesDF = spark.read.option("sep", "\t").schema(schema)\
    .csv(f"file:///{os.path.abspath('')}/../data/ml-100k/u.data")

movieCounts = moviesDF.groupBy("movieID").count()
movieCounts

DataFrame[movieID: int, count: bigint]

In [7]:

# Create a user-defined function to look up movie names from our broadcasted dictionary
def lookupName(movieID):
    return nameDict.value[movieID]

lookupNameUDF = sf.udf(lookupName)

In [10]:
# Add a movieTitle column using our new udf
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(sf.col("movieID")))
moviesWithNames.show(10)

+-------+-----+--------------------+
|movieID|count|          movieTitle|
+-------+-----+--------------------+
|    496|  231|It's a Wonderful ...|
|    471|  221|Courage Under Fir...|
|    463|   71|Secret of Roan In...|
|    148|  128|Ghost and the Dar...|
|   1342|    2|Convent, The (Con...|
|    833|   49|  Bulletproof (1996)|
|   1088|   13|  Double Team (1997)|
|   1591|    6|Duoluo tianshi (1...|
|   1238|    8|   Full Speed (1996)|
|   1580|    1|     Liebelei (1933)|
+-------+-----+--------------------+
only showing top 10 rows



In [11]:
# Sort the results
sortedMoviesWithNames = moviesWithNames.orderBy(sf.desc("count"))

# Grab the top 10
sortedMoviesWithNames.show(10, False)

+-------+-----+-----------------------------+
|movieID|count|movieTitle                   |
+-------+-----+-----------------------------+
|50     |583  |Star Wars (1977)             |
|258    |509  |Contact (1997)               |
|100    |508  |Fargo (1996)                 |
|181    |507  |Return of the Jedi (1983)    |
|294    |485  |Liar Liar (1997)             |
|286    |481  |English Patient, The (1996)  |
|288    |478  |Scream (1996)                |
|1      |452  |Toy Story (1995)             |
|300    |431  |Air Force One (1997)         |
|121    |429  |Independence Day (ID4) (1996)|
+-------+-----+-----------------------------+
only showing top 10 rows

